In [1]:
!pip install pip install protobuf==3.20.*

/bin/bash: /home/therandomtroll/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
import random

import tensorflow as tf
import glob
from sklearn import metrics
from tensorflow.keras import utils

from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from tensorflow.keras.models import load_model
from matplotlib import pyplot
from numpy import expand_dims
from matplotlib.patches import Rectangle

import warnings
warnings.filterwarnings('ignore')

tf.compat.v1.disable_eager_execution()

In [ ]:
label_df = pd.read_csv(f'FullIJCNN2013/gt.txt',names=['file','x1','y1','x2','y2','class'],delimiter = ';')
label_df

In [ ]:
#blue circle signs
zero = [33,34,35,36,37,38,39,40]
#b/w stripe circle
one = [6,32,41,42]
#danger
two = [17]
#diamond
three = [12]
#red circle
four = [0,1,2,3,4,5,7,8,9,10,15,16]
#red down triangle
five = [13]
#red up triangle
six = [11,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
#stop
seven = [14]

subclass_list = [zero, one, two, three, four, five, six, seven]

In [ ]:
# implement image subclass from above
def find_subclass(classid):
    for i in range(8):
        if classid in subclass_list[i]:
            return i
label_df['sub_class']=label_df['class'].map(find_subclass)
label_df

In [ ]:
for j in range(900):
    image_id = str(j).rjust(5,'0')
    f_out = open(f'FullIJCNN2013/xml/{image_id}.xml','w')
    f_out.write(f"""<annotation>
    <folder>{f'FullIJCNN2013/'}</folder>
    <path>{f'FullIJCNN2013/{image_id}.ppm'}</path>
    <source>
        <database>Unknown</database>
    </source>
    <filename>{str(j).rjust(5,'0')}.ppm</filename>
    <size>
        <width>1360</width>
        <height>800</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
""")
    temp_df = label_df[label_df['file'] == image_id+'.ppm']
    for i in range(temp_df.shape[0]):
        f_out.write(f"""    <object>
        <name>{temp_df.iloc[i,6]+1}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{temp_df.iloc[i,1]}</xmin>
            <ymin>{temp_df.iloc[i,2]}</ymin>
            <xmax>{temp_df.iloc[i,3]}</xmax>
            <ymax>{temp_df.iloc[i,4]}</ymax>
        </bndbox>
    </object>
""")        
    f_out.write("</annotation>")
    f_out.close()

In [ ]:
class RoadDataset(Dataset):
    #Load the model
    #is_train loads the set type, dataset_dir is where images are
    #train_list and test_list to differntiate which file is what set
    def load_dataset(self, dataset_dir,is_train=True):
        #Two categories, normal and not
        for i in range(8):
            self.add_class('dataset',i+1,str(i+1))
        files = glob.glob(dataset_dir + 'normalized/*.ppm')
        for file in files:
            image_idstr = file[-14:-9]
            image_id = int(image_idstr)
            if is_train and image_id >= 600:
                continue
            elif not is_train and image_id < 600:
                continue
            img_path = file
            ann_path = f'{dataset_dir}xml/{image_idstr}.xml'
            # add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, 
                       annotation=ann_path,class_ids=[0,1,2])
        
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        # print(boxes)
        return boxes, width, height

    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load XML
        boxes, w, h = self.extract_boxes(path)
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = int(box[4])
            class_ids.append(self.class_names.index(str(box[4])))
        return masks, asarray(class_ids, dtype='int32')

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
# define a configuration for the model
class RoadConfig(Config):
    # define the name of the configuration
    NAME = "road_cfg"
    # number of classes (background + num signs) (8 for sub cats, 43 for all signs)
    NUM_CLASSES = 1 + 8
    DETECTION_MIN_CONFIDENCE = 0.5
    # number of training steps per epoch
    STEPS_PER_EPOCH = 131

In [ ]:
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "road_cfg"
    # number of classes (background + num signs)
    NUM_CLASSES = 1 + 8
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.1

In [ ]:
#make train dataset
train_set = RoadDataset()
train_set.load_dataset('FullIJCNN2013/',is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

In [ ]:
#make test dataset
test_set = RoadDataset()
test_set.load_dataset('FullIJCNN2013/',is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
config = RoadConfig()
config.display()

In [ ]:
model = MaskRCNN(mode='training', model_dir='../models', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5',
                   by_name=True
                   ,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                            "mrcnn_bbox", "mrcnn_mask"]
                  )

In [ ]:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=45, layers='heads')